
# Space

In [1]:
import os
import sys 
import logging
import pandas as pd 
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import pandas as pd
from datetime import datetime 

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


/Users/floydluo/Library/CloudStorage/OneDrive-DRFIRST.COM,INC/Documents - DrFirst-JHU Collaboration Center/DrFirst-Project/2024-DrFirst-v2-SPACE/_DrFirst-AI-EduRxPred-WorkSpace
{'CODE_FN': '../pipeline',
 'CODE_RFT': '../pipeline',
 'DATA_CaseFeat': '../_Data/3-Data_CaseFeat',
 'DATA_CaseObs': '../_Data/2-Data_CaseObs',
 'DATA_CaseSet': '../_Data/4-Data_CaseSet',
 'DATA_EXTERNAL': '../external',
 'DATA_RAW': '../_Data/0-Data_Raw',
 'DATA_RFT': '../_Data/1-Data_RFT',
 'DATA_TASK': 'Data_EduRxPred',
 'MODEL_TASK': 'Model_EduRxPred',
 'TaskName': 'EduRxPred',
 'WORKSPACE_PATH': '/Users/floydluo/Library/CloudStorage/OneDrive-DRFIRST.COM,INC/Documents '
                   '- DrFirst-JHU Collaboration '
                   'Center/DrFirst-Project/2024-DrFirst-v2-SPACE/_DrFirst-AI-EduRxPred-WorkSpace'}


# Dataset

In [2]:
import datasets
from recfldtkn.loadtools import load_ds_rec_and_info
from recfldtkn.configfn import load_cohort_args, load_record_args

base_config = load_cohort_args(recfldtkn_config_path, SPACE)
print(base_config)

{'CohortInfo': {'RawData2023Nov_Trulicity': {'cohort_label': 1, 'cohort_name': 'RawData2023Nov_Trulicity', 'FolderPath': '../_Data/0-Data_Raw/2023_11_30_30days_Trulicity'}}, 'RawRootID': 'patient_id_encoded', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_Sequence': ['P', 'PInv', 'Rx', 'EgmAuthen', 'EgmCallPharm', 'EgmClick', 'EgmCopay', 'EgmEdu', 'EgmRmd'], 'RecName_to_PrtRecName': {'P': 'None', 'PInv': 'P', 'Rx': 'PInv', 'EgmAuthen': 'PInv', 'EgmCallPharm': 'PInv', 'EgmClick': 'PInv', 'EgmCopay': 'Rx', 'EgmEdu': 'Rx', 'EgmRmd': 'Rx'}, 'RecName_to_RFT_GROUP_SIZE': {'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'Default': 10000}, 'RecName_to_RFT_usebucket': {'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_path': '../pipeline/config_r

In [3]:
from recfldtkn.loadtools import fetch_trigger_tools
from recfldtkn.loadtools import fetch_casetag_tools, fetch_casefilter_tools


def get_Trigger_Cases(TriggerCaseMethod, 
                      cohort_label_list, 
                      base_config, 
                      SPACE, 
                      RecName_to_dsRec = {},
                      RecName_to_dsRecInfo = {}):
    
    Trigger_Tools = fetch_trigger_tools(TriggerCaseMethod, SPACE)
    case_id_columns = Trigger_Tools['case_id_columns']
    special_columns = Trigger_Tools['special_columns'] 
    TriggerRecName = Trigger_Tools['TriggerRecName']
    convert_TriggerEvent_to_Caseset = Trigger_Tools['convert_TriggerEvent_to_Caseset']
    ###########################
    if TriggerRecName in RecName_to_dsRec:
        ds_rec = RecName_to_dsRec[TriggerRecName]
    else:
        ds_rec, _ = load_ds_rec_and_info(TriggerRecName, base_config, cohort_label_list)
    df_case = convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config)
    ###########################
    return df_case

In [4]:
def convert_TriggerCases_to_LearningCases(df_case, 
                                          cohort_label_list,
                                          Trigger2LearningMethods, 
                                          base_config, 
                                          use_inference):
    if use_inference == True:
        Trigger2LearningMethods = [i for i in Trigger2LearningMethods if i['type'] != 'learning-only']
    else:
        Trigger2LearningMethods = [i for i in Trigger2LearningMethods if i['type'] != 'inference-only']
        
    # print(Trigger2LearningMethods)
    for method in Trigger2LearningMethods:
        if method['op'] == 'Tag':
            name = method['Name']
            logger.info(f'CaseTag: {name}')
            CaseTag_Tools = fetch_casetag_tools(name, SPACE)

            subgroup_columns = CaseTag_Tools['subgroup_columns']
            if 'InfoRecName' in CaseTag_Tools:
                InfoRecName = CaseTag_Tools['InfoRecName']
                ds_info, _ = load_ds_rec_and_info(InfoRecName, base_config, cohort_label_list)
            else:
                ds_info = None

            fn_case_tagging = CaseTag_Tools['fn_case_tagging']
            df_case = fn_case_tagging(df_case, ds_info, subgroup_columns, base_config)

        elif method['op'] == 'Filter':
            name = method['Name']
            logger.info(f'CaseFilter: {name}')
            CaseFilter_Tools = fetch_casefilter_tools(name, SPACE)
            fn_case_filtering = CaseFilter_Tools['fn_case_filtering']
            
            logger.info(f'Before Filter: {df_case.shape}')
            df_case = fn_case_filtering(df_case)
            logger.info(f'After Filter: {df_case.shape}')

        else:
            raise ValueError(f'Unknown method: {method}')

    return df_case

# TriggerCaseMethod

In [5]:
use_learning = True 
use_inference = not use_learning

######################################
TriggerCaseMethod = 'TrulicityRx'
cohort_label_list = [1]
Trigger2LearningMethods = [
    {'op':'Tag',    'Name': 'TagPttBasicInfo', 'type': 'learning-only'},
    {'op':'Filter', 'Name': 'FilterBasicPRx',  'type': 'learning-only'},
]
######################################

In [6]:
use_inference

False

In [7]:
RecName_to_dsRec = {}
RecName_to_dsRecInfo = {}

df_case = get_Trigger_Cases(TriggerCaseMethod, 
                            cohort_label_list, 
                            base_config, 
                            SPACE, 
                            RecName_to_dsRec, 
                            RecName_to_dsRecInfo)

logger.info(f'Before: {df_case.shape}')
df_case = convert_TriggerCases_to_LearningCases(df_case, 
                                                cohort_label_list,
                                                Trigger2LearningMethods, 
                                                base_config, 
                                                use_inference)
logger.info(f'After: {df_case.shape}')

[INFO:2024-03-03 02:33:23,692:(1623119124.py@11 __main__)]: Before: (101006, 6)
[INFO:2024-03-03 02:33:23,693:(2386204352.py@15 __main__)]: CaseTag: TagPttBasicInfo
[INFO:2024-03-03 02:33:23,746:(2386204352.py@30 __main__)]: CaseFilter: FilterBasicPRx
[INFO:2024-03-03 02:33:23,748:(2386204352.py@34 __main__)]: Before Filter: (101006, 10)
[INFO:2024-03-03 02:33:23,792:(2386204352.py@36 __main__)]: After Filter: (98762, 10)
[INFO:2024-03-03 02:33:23,794:(1623119124.py@17 __main__)]: After: (98762, 10)


# Split

In [8]:
df_case # df_case_learning

,PID,ObsDT,PInvID,RxID,DT,drug_name,patient_gender,patient_age_bucket,patient_zipcode_3,cohort
0,1000001,2023-09-07 18:13:08.746,1000001-005,1000001-005-000,2023-09-07 18:13:08.746,Trulicity,M,41-50,494.0,C1
1,1000002,2023-09-21 23:38:11.348,1000002-000,1000002-000-000,2023-09-21 23:38:11.348,Trulicity,F,51-60,350.0,C1
2,1000003,2023-09-05 16:33:52.699,1000003-000,1000003-000-000,2023-09-05 16:33:52.699,Trulicity,F,71-80,213.0,C1
3,1000004,2023-05-09 20:46:13.911,1000004-003,1000004-003-000,2023-05-09 20:46:13.911,Trulicity,F,71-80,497.0,C1
4,1000004,2023-09-01 17:07:57.276,1000004-007,1000004-007-000,2023-09-01 17:07:57.276,Trulicity,F,71-80,497.0,C1
...,...,...,...,...,...,...,...,...,...,...
98757,1046208,2023-09-12 16:09:16.572,1046208-000,1046208-000-001,2023-09-12 16:09:16.572,Trulicity,M,31-40,735.0,C1
98758,1046209,2023-05-04 13:45:40.707,1046209-000,1046209-000-000,2023-05-04 13:45:40.707,Trulicity,M,51-60,782.0,C1
98759,1046209,2023-07-18 15:48:38.928,1046209-002,1046209-002-000,2023-07-18 15:48:38.928,Trulicity,M,51-60,782.0,C1
98760,1046209,2023-07-20 15:05:52.683,1046209-004,1046209-004-000,2023-07-20 15:05:52.683,Trulicity,M,51-60,782.0,C1


In [9]:
from recfldtkn.pipeline_model import generate_random_tags, assign_caseSplitTag_to_dsCaseLearning

##############
RANDOM_SEED = 42
downsample_ratio = 1 # 1 (don't drop any case), 0.1 (drop 90% of cases of one patient).
out_ratio = 0 # hold-out patients
test_ratio = '2023.10.15'#  'tail0.1' # '0.1'
valid_ratio = 0.1 
##############

SplitMethod = f'rs{RANDOM_SEED}-ds{downsample_ratio}-out{out_ratio}ts{test_ratio}vd{valid_ratio}' 
print(SplitMethod)

rs42-ds1-out0ts2023.10.15vd0.1


In [10]:
logger.info('Generate Random Tags for Downsample, In/Out, and Train/Test/Valid Split')
logger.info(f'SplitMethod: {SplitMethod}')
logger.info(f'Before Split: {df_case.shape}')
df_case = assign_caseSplitTag_to_dsCaseLearning(df_case, 
                                                RANDOM_SEED, 
                                                downsample_ratio, out_ratio, 
                                                test_ratio, valid_ratio)
logger.info(f'After Split: {df_case.shape}')
df_case.head()

[INFO:2024-03-03 02:33:29,722:(3087417902.py@1 __main__)]: Generate Random Tags for Downsample, In/Out, and Train/Test/Valid Split
[INFO:2024-03-03 02:33:29,722:(3087417902.py@2 __main__)]: SplitMethod: rs42-ds1-out0ts2023.10.15vd0.1
[INFO:2024-03-03 02:33:29,723:(3087417902.py@3 __main__)]: Before Split: (98762, 10)
[INFO:2024-03-03 02:33:29,865:(3087417902.py@8 __main__)]: After Split: (98762, 15)


,PID,ObsDT,PInvID,RxID,DT,drug_name,patient_gender,patient_age_bucket,patient_zipcode_3,cohort,Out,In,Test,Valid,Train
0,1015641,2023-03-17 15:20:48.692,1015641-000,1015641-000-000,2023-03-17 15:20:48.692,Trulicity,F,51-60,350.0,C1,False,True,False,False,True
1,1026251,2023-09-20 23:28:54.737,1026251-000,1026251-000-000,2023-09-20 23:28:54.737,Trulicity,F,31-40,606.0,C1,False,True,False,False,True
2,1001401,2023-10-28 00:58:29.134,1001401-031,1001401-031-001,2023-10-28 00:58:29.134,Trulicity,F,81+,856.0,C1,False,True,True,False,False
3,1019852,2023-09-01 21:12:22.742,1019852-020,1019852-020-000,2023-09-01 21:12:22.742,Trulicity,M,51-60,900.0,C1,False,True,False,False,True
4,1045643,2023-08-23 19:38:28.001,1045643-013,1045643-013-000,2023-08-23 19:38:28.001,Trulicity,F,41-50,741.0,C1,False,True,False,False,True


In [11]:
print(f'\ntotal---> recnum {len(df_case)}')
for i in ['In', 'Out', 'Train', 'Valid', 'Test']:
    print(i, df_case[i].mean())

print('Valid/(Train+Test):', df_case['Valid'].sum() / (df_case['Train'].sum() + df_case['Valid'].sum()))

subtype_list = ['patient_gender', 'patient_age_bucket', 'patient_zipcode_3']
subtype_list = ['patient_gender', 'patient_age_bucket']
for subtype in subtype_list:
    for subname, df_sub in df_case.groupby(subtype):
        print(f'\n{subname}---> recnum {len(df_sub)}')
        for i in ['In', 'Out', 'Train', 'Valid', 'Test']:
            print(i, df_sub[i].mean())
        print('Valid/(Train+Test):', df_sub['Valid'].sum() / (df_sub['Train'].sum() + df_sub['Valid'].sum()))


total---> recnum 98762
In 1.0
Out 0.0
Train 0.8259755776513233
Valid 0.09353800044551548
Test 0.08048642190316113
Valid/(Train+Test): 0.10172552387873983

F---> recnum 56960
In 1.0
Out 0.0
Train 0.8236130617977528
Valid 0.09322331460674158
Test 0.08316362359550562
Valid/(Train+Test): 0.10167933669073013

M---> recnum 41793
In 1.0
Out 0.0
Train 0.8291579929653291
Valid 0.09398703132103463
Test 0.07685497571363625
Valid/(Train+Test): 0.10181177263419817

18-30---> recnum 2559
In 1.0
Out 0.0
Train 0.8096912856584604
Valid 0.09886674482219616
Test 0.0914419695193435
Valid/(Train+Test): 0.10881720430107526

31-40---> recnum 7804
In 1.0
Out 0.0
Train 0.8145822655048693
Valid 0.09238851870835468
Test 0.09302921578677602
Valid/(Train+Test): 0.10186493359706132

41-50---> recnum 17244
In 1.0
Out 0.0
Train 0.8235328230109024
Valid 0.09098816979819067
Test 0.08547900719090698
Valid/(Train+Test): 0.09949270767279644

51-60---> recnum 29655
In 1.0
Out 0.0
Train 0.8242117686730737
Valid 0.095228460